In [6]:
import altair as alt
import pandas as pd
from IPython.display import display

nvidia_csv_path = "./benchmark4090.csv"
mac_csv_path = "./benchmarkMac.csv"

def load_data(path):
    csv_data = pd.read_csv(path)
    csv_data = csv_data[(csv_data["InputWidth"] == 1920)]
    csv_data["ComputeConversionTimeMicros"] = csv_data["ComputeConversionTimeMicros"] + csv_data["TransferDeviceVisibleToDeviceLocalTimeMicros"]
    for time_column in ["TotalTime", "CopyToDeviceVisibleTimeMicros", "ComputeConversionTimeMicros", "CopyDeviceVisibleToHostLocalTimeMicros"]:
       csv_data[time_column] = csv_data[time_column].div(1000)
    return csv_data
nvidia_data = load_data(nvidia_csv_path)
mac_data = load_data(mac_csv_path)

In [7]:
def create_chart(df, name):
    input_formats = df["InputFormat"].unique()
    input_formats_dropdown = alt.binding_select(options=input_formats, name="Input format")
    input_formats_select = alt.selection_single(
        fields=["InputFormat"],
        bind=input_formats_dropdown,
        name="Input format",
        init={'InputFormat': input_formats[1]}
    )

    output_formats = df["OutputFormat"].unique()
    output_formats_dropdown = alt.binding_select(options=output_formats, name="Output format")
    output_formats_select = alt.selection_single(
        fields=["OutputFormat"],
        bind=output_formats_dropdown,
        name="Output format",
        init={'OutputFormat': output_formats[1]}
    )
    
    columns = [
        "CopyToDeviceVisibleTimeMicros", 
        "ComputeConversionTimeMicros",
        "CopyDeviceVisibleToHostLocalTimeMicros"
    ]

    columns_names = [
        "Device visible copy",
        "Kernel execution",
        "Readback"
    ]
    
    for name, title in zip(columns, columns_names):
        df[title] = df[name]

    base = alt.Chart(df).mark_area(size=1).encode(
        x=alt.X("OutputHeight:Q", title="Resolution (Height, 16:9)"),
        y=alt.Y("TotalTime:Q", title="Time (ms)"),
        opacity=alt.value(0.3)
    )
    
    lines = alt.Chart(df).transform_fold(
        fold=columns_names
    ).mark_line().encode(
        x='OutputHeight:Q',
        y='value:Q',
        color=alt.Color('key:N', title="Total time",legend=alt.Legend(
                orient='none',
                legendX=10, legendY=30,
                direction='vertical',))
    )

    return alt.layer(
        *[base, lines]
    ).resolve_scale(
        y="shared",
    ).add_selection(
        input_formats_select, output_formats_select
    ).transform_filter(
        input_formats_select & output_formats_select
    ).properties(
    ).interactive()

nvidia_chart = create_chart(nvidia_data, "Nvidia")
mac_chart = create_chart(mac_data, "Mac")

alt.concat(nvidia_chart, mac_chart).resolve_scale(y="shared", color='independent')

alt.ConcatChart(...)

In [8]:
libraries = ["PIXELWEAVE", "LIBYUV", "ZIMG", "SWS"]

def create_comparison_chart(df, max_value, step):
    return alt.Chart(df).mark_bar().encode(
        alt.X('Library', sort=libraries),
        alt.Y('ConversionTimeMicros:Q', title="Time (ms)", axis=alt.Axis(values=list(range(0, max_value, step)))),
        color='Library'
    ).properties(
        width=100,
        height=150
    )

uyvy_data_windows = pd.DataFrame(list(zip(libraries, [6670/1000, 13836/1000, 7855/1000, 18267/1000])), columns=["Library", "ConversionTimeMicros"])
uyvy_windows_chart = create_comparison_chart(uyvy_data_windows, 20, 5)

uyvy_data_mac = pd.DataFrame(list(zip(libraries, [4610/1000, 8453/1000, 4167/1000, 5120/1000])), columns=["Library", "ConversionTimeMicros"])
uyvy_mac_chart = create_comparison_chart(uyvy_data_mac, 20, 5)

bgra_data_windows = pd.DataFrame(list(zip(libraries, [5544 / 1000,8918 / 1000, 10455 / 1000, 12705 / 1000])), columns=["Library", "ConversionTimeMicros"])
bgra_windows_chart = create_comparison_chart(bgra_data_windows, 14, 2)

bgra_data_mac = pd.DataFrame(list(zip(libraries, [6086/1000, 8297/1000, 5107/1000, 5623/1000])), columns=["Library", "ConversionTimeMicros"])
bgra_mac_chart = create_comparison_chart(bgra_data_mac, 14, 2)

display(alt.vconcat(alt.concat(uyvy_windows_chart, uyvy_mac_chart).resolve_scale(y="shared"), alt.concat(bgra_windows_chart, bgra_mac_chart).resolve_scale(y="shared")))

alt.VConcatChart(...)

In [11]:
data = [
    ["Nvidia 4090", "Host memory", 52.3266, 50279/1000],
    ["Nvidia 4090", "Staging buffer", 0.661612, 371/1000],
    ["Apple M2", "Host memory", 1.13086, 780/1000],
    ["Apple M2", "Staging buffer", 1.6788, 1357/1000],
]

memory = pd.DataFrame(data, columns=["GPU", "Memory allocation strategy", "Total time (ms)", "Compute time (ms)"])
chart = alt.Chart(memory).mark_bar(
    opacity=1,
    ).encode(
    column = alt.Column('GPU', title="", header=alt.Header(labelOrient = "top")),
    x =alt.X('Memory allocation strategy', title="", axis=alt.Axis(labels=False)),
    y =alt.Y('Total time (ms):Q', scale=alt.Scale(type='sqrt')),
    color=alt.Color('Memory allocation strategy')).properties(
        width=100,
        height=150
    )
display(chart)


alt.Chart(...)